In [ ]:
# default_exp checker

# Dependency Checker

> A pragmatic way to talk with pypi and find out what dependencies are out of date

In [ ]:
#hide
from nbdev.showdoc import *

## Dependency Traversing

Sometimes, we may want to check the current installed versions of a project's basic dependencies, and further check if those dependencies are out of date. `dependency_checker` is designed around this concept, utilizing the `pipdeptree` library.

In [ ]:
#export
import json, ast, pipdeptree, sys, subprocess
from github import Github

import requests

from packaging.version import parse
import sys

if sys.version_info < (3, 8):
    import importlib_metadata
else:
    import importlib.metadata as importlib_metadata

/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
#export
def get_installed_dependencies(
    package_name:str, # The name of a python package
    depth_limit:int=1, # How deep to follow nested dependencies
    include_self:bool=False, # Whether to include the original library in the results
) -> dict: # A dictionary of {package:version}
    "Recursively grabs dependencies of python package"
    pkgs = pipdeptree.get_installed_distributions(local_only=False, user_only=False)
    tree = pipdeptree.PackageDAG.from_pkgs(pkgs)
    tree = tree.filter([package_name], None)
    curr_depth=0
    def _get_deps(j, dep_dict={}, curr_depth=0):
        if curr_depth > depth_limit: return dep_dict
        if isinstance(j, list):
            for a in j:
                _get_deps(a, dep_dict, curr_depth)
        elif isinstance(j, dict):
            if 'package_name' in j.keys():
                if j['package_name'] not in dep_dict.keys():
                    dep_dict[j['package_name']] = j['installed_version']
            if 'dependencies' in j.keys():
                curr_depth += 1
                return _get_deps(j['dependencies'], dep_dict, curr_depth)
        return dep_dict
    deps = _get_deps(ast.literal_eval(pipdeptree.render_json_tree(tree, 4)), {})
    if not include_self: deps.pop(package_name, None)
    return deps

This function operates by traversing a DAG and grabbing dependencies of projects found from it. Generally a depth of 1 is recommended, below is a quick guide to what will be returned at each depth.


**0**: A depth of zero will an empty dictionary  unless `include_self` is `True`. If so, it will include only the library name:

In [ ]:
deps = get_installed_dependencies('pipdeptree', depth_limit=0)

In [ ]:
assert deps == {}

In [ ]:
deps = get_installed_dependencies('pipdeptree', depth_limit=0, include_self=True)

In [ ]:
assert deps == {'pipdeptree':'2.2.1'}

**1**: A depth of one will return the project and its main dependencies (if `include_self` is `True`), such as those stated in the `requirements.txt` as well as packages such as `pip` 

In [ ]:
deps = get_installed_dependencies('pipdeptree', depth_limit=1, include_self=True)

In [ ]:
assert len(deps.keys()) == 2

In [ ]:
assert all(package in deps.keys() for package in ('pipdeptree', 'pip'))

In [ ]:
deps = get_installed_dependencies('pipdeptree', depth_limit=1, include_self=False)

In [ ]:
assert len(deps.keys()) == 1

In [ ]:
assert 'pip' in deps.keys()

**2+**: A depth of two or greater will return the dependencies for each of the dependencies above that layer. These allow for more fine-grained requirements

## Checking for New Versions

Given these dependencies, we can also then check for a new version to see if an upgrade is available. This is what the `is_latest_version` function is designed for:

In [ ]:
#export
def is_latest_version(
    package_name:str, # The name of a pip python package 
    current_version:str, # The installed version of a package, such as "1.2.3"
) -> bool: # Whether the versions are the same
    "Compares the current version with the latest version, and returns if they are different"
    latest_version = str(subprocess.run([sys.executable, '-m', 'pip', 'install', '{}==random'.format(package_name)], capture_output=True, text=True))
    latest_version = latest_version[latest_version.find('(from versions:')+15:]
    latest_version = latest_version[:latest_version.find(')')]
    latest_version = latest_version.replace(' ','').split(',')[-1]

    if latest_version == current_version:
        return True
    else:
        return False

In [ ]:
using_latest_version = is_latest_version('pipdeptree', '2.0.9')
assert using_latest_version == False

Here we tested if `pipdeptree` is the latest version. The version we specified is one less than that of the latest release at the time of development. We got `False`, meaning a newer version is available.

### Checking the Release Notes

In [ ]:
#export
def get_github_repo_from_pypi(
    name:str # The name of a pypi package
):
    "Gets the Github URL for a pypi package"
    url = f"https://pypi.org/pypi/{name}/json"
    r = requests.get(url)
    urls = r.json()["info"]["project_urls"]
    for url in urls.values():
        if "github.com" in url:
            return "/".join(url.split("github.com/")[1].split("/")[:2])
    return ""

In [ ]:
assert get_github_repo_from_pypi("dependency_checker") == "muellerzr/dependency_checker"

In [ ]:
#export
def get_latest_release_notes(
    package:str # The name of a pypi package
):
    "Gets the URL for the latest release notes of a pypi package from Github if available"
    client = Github()
    url = get_github_repo_from_pypi(package)
    if url != "":
        repo = client.get_repo(url)
        try:
            latest = repo.get_latest_release()
            notes_url = latest.html_url
            release_tag = latest.tag_name
            return {"notes_url":notes_url, "release_tag":release_tag}
        except Exception as e:
            if e.args[0] == 404:
                return {}
            raise
    return {}

In [ ]:
assert get_latest_release_notes("dependency_checker") == {}
assert list(get_latest_release_notes("torch").keys()) == ["notes_url", "release_tag"]

In [ ]:
def _in_notebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

In [ ]:
#export
def check_for_newer_release(
    package:str, # The name of a pypi package
    version:str = None # An optional version number
) -> bool:
    "Checks if a newer release is available and then finds the release notes"
    if version is None:
        version = parse(importlib_metadata.version(package))
    if not is_latest_version(package, str(version)):
        notes = get_latest_release_notes(package)
        if notes != {}:
            s = f"Newer version of `{package}` was found available on pypi ({str(version)} -> {notes['release_tag']})\n\n"
            s += f"To upgrade run `pip install {package} -U`\n\n"
            if _in_notebook():
                from IPython.display import display, Markdown
                s += f"[Click Here]({notes['notes_url']}) to see the latest release notes."
                display(Markdown(s))
            else:
                s += f"To read the latest release notes go to: {notes['notes_url']}"
                print(s)
                return True
        else:
            return False
    return False

With this utility we can quickly get a pretty formatted print of the new version to install, how to install it, and the latest release notes as a link:

In [ ]:
check_for_newer_release("torch", "1.9.0")

Newer version of `torch` was found available on pypi (1.9.0 -> v1.11.0)

To upgrade run `pip install torch -U`

[Click Here](https://github.com/pytorch/pytorch/releases/tag/v1.11.0) to see the latest release notes.

False